## Convert DCM images to PNG

In [25]:
import concurrent.futures
import pandas as pd
import os
from PIL import Image

In [26]:
train_data = pd.read_csv('../../data/train/data.csv')
test_data = pd.read_csv('../../data/test/data.csv')
png_images_path = '../../data/RSNA_BME548/images/'

In [27]:
# create train and test folders inside the images folder
train_images_path = '../../data/RSNA_BME548-png/train/'
test_images_path = '../../data/RSNA_BME548-png/test/'

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(test_images_path, exist_ok=True)

# create subfolders for each class
for class_name in train_data['any'].unique():
    os.makedirs(train_images_path + str(class_name), exist_ok=True)
    os.makedirs(test_images_path + str(class_name), exist_ok=True)

In [28]:
train_files = train_data["filename"].values
test_files = test_data["filename"].values

In [29]:
def search_file(directory, search_string):
    """
    Searches for files whose filename contains the specified search string
    in the given directory and its subdirectories.
    """
    matches = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if search_string in file:
                file_path = os.path.join(root, file)
                matches.append(file_path)
    return matches

In [30]:
def process_file(file, train):
    images_path = train_images_path if train else test_images_path
    data = train_data if train else test_data

    matches = search_file(png_images_path, file)
    if len(matches) == 0:
        print(f"File {file} not found.")
        return
    match = matches[0]
    image = Image.open(match)
    image = image.resize((224, 224))
    new_file = file.replace('.dcm', '.png')
    image.save(images_path + str(data[data['filename'] == file]['any'].values[0]) + '/' + new_file)

with concurrent.futures.ThreadPoolExecutor() as executor:
    for file in train_files:
        executor.submit(process_file, file, True)
    for file in test_files:
        executor.submit(process_file, file, False)

File ID_002697199.dcm not found.


In [ ]:
# Resize images to 224x224
for class_name in train_data['any'].unique():
    for file in os.listdir(train_images_path + str(class_name)):
        image = Image.open(train_images_path + str(class_name) + '/' + file)
        image = image.resize((224, 224))
        image.save(train_images_path + str(class_name) + '/' + file)

        